### PD3
##### Artur Żółkowski

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
np.random.seed = 42

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/mini-pw/2021L-WUM/main/Prace_domowe/Praca_domowa3/australia.csv")
df.head()

In [ ]:
df.hist(figsize=(20,15))
plt.show()

In [ ]:
X = df.drop("RainTomorrow", axis=1)
y = df[["RainTomorrow"]]

In [ ]:
y.describe()

Jak można zauważyć mamy znacznie więcej dni bez deszczu niż tych deszczowych. By zachować odpowiednią proporcję tych dni w zbiorze testowym i treningowym, przy podziale użyjemy stratify.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
kfolds = 3
split = KFold(n_splits=kfolds, shuffle=True, random_state=42)
indicators = ["recall", "accuracy", "roc_auc", "f1"]
models_summary = pd.DataFrame(columns = indicators) 

## Modele

### Random forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=400, max_depth=40, max_features=4, random_state=42)
rf_model = Pipeline(steps=[('standardscaler', StandardScaler()), ('rf', rfc)])
    
rf_cv_results = cross_validate(rf_model, X_train, y_train, cv=split, scoring=indicators, n_jobs=-1)

In [ ]:
models_summary.loc["RF"]=[np.mean(rf_cv_results["test_recall"]), np.mean(rf_cv_results["test_accuracy"]), 
                          np.mean(rf_cv_results["test_roc_auc"]), np.mean(rf_cv_results["test_f1"])]

In [ ]:
models_summary.loc["RF"]

### SVC

In [ ]:
svc = svm.SVC(kernel="linear", random_state=42)
svc_model = Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', svc)])

sv_cv_results = cross_validate(svc_model, X_train, y_train, cv=split, scoring=indicators, n_jobs=-1)

In [ ]:
models_summary.loc["SVC"]=[np.mean(sv_cv_results["test_recall"]), np.mean(sv_cv_results["test_accuracy"]), 
                           np.mean(sv_cv_results["test_roc_auc"]), np.mean(sv_cv_results["test_f1"])]

In [ ]:
models_summary.loc["SVC"]

### Logistic regression

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2, n_jobs=-1)
knn_model = Pipeline(steps=[('standardscaler', StandardScaler()), ('knn', knn)])

knn_cv_results = cross_validate(knn_model, X_train, y_train, cv=split, scoring=indicators, n_jobs=-1)

In [ ]:
models_summary.loc["KNN"]=[np.mean(knn_cv_results["test_recall"]), np.mean(knn_cv_results["test_accuracy"]), 
                           np.mean(knn_cv_results["test_roc_auc"]), np.mean(knn_cv_results["test_f1"])]

In [ ]:
models_summary.loc["KNN"]

## Porównanie modeli

In [ ]:
models_summary.reset_index()

In [ ]:
models_summary.T.plot.bar()
plt.show()

Porównując wytrenowane modele możemy zauważyć, że las losowy wypada najlepiej w każdej z obliczonych miar. Oczywiście dobierając nieco inne wartości hiperparametrów wyniki mogłyby być różne. 
Porównując SVC z KNN możemy zauważyć, że ten drugi ma wyższy wynik recall, jednak wypada znacznie słabiej w pozostałych wartościach. Jeśli jednak chcielibyśmy przewidzieć jak najwięcej deszczowych dni mógłby być on warty rozważenia. 
Możemy zauważyć, że wszystkie modele mają wysokie accuracy, jednak jeśli weźmiemy pod uwagę, że dni bez deszczu stanowią zdecydowaną większość, wówczas wartości te nie robią żadnego wrażenia. Model który za każdym razem przewidywałby, że nie będzie deszczu, miałby równie wysokie accuracy, jednakże oczywiście byłby bezużyteczny.